Esta nota utiliza métodos vistos en [1.5.Integracion_numerica](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/I.computo_cientifico/1.5.Integracion_numerica.ipynb)

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_numerical -p 8888:8888 -p 8786:8786 -p 8787:8787 -d palmoreck/jupyterlab_numerical:1.1.0
```

Detener el contenedor de docker:

```
docker stop jupyterlab_local
```


Instalamos las herramientas que nos ayudarán al perfilamiento:

In [1]:
%pip install -q --user line_profiler

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -q --user memory_profiler

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -q --user psutil

Note: you may need to restart the kernel to use updated packages.


La siguiente celda reiniciará el kernel de **IPython** para cargar los paquetes instalados en la celda anterior. Dar **Ok** en el mensaje que salga y continuar con el contenido del notebook.

In [4]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import math
import numpy as np

from scipy.integrate import quad
import matplotlib.pyplot as plt

# Perfilamiento en Python

En esta nota revisamos algunas herramientas de Python para perfilamiento de código: uso de cpu y memoria

Medición de tiempos con:

* Módulo [time](https://docs.python.org/3/library/time.html#time.time) de Python.

* [%time](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-time) de comandos de magic. Esta es sólo para medir tiempos de un statement.

* [/usr/bin/time](https://en.wikipedia.org/wiki/Time_(Unix)) de `Unix`.

* [%timeit](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit) de comandos de magic.

Perfilamiento:

* De CPU con: [line_profiler](https://pypi.org/project/line-profiler/), [CProfile](https://docs.python.org/2/library/profile.html) que es `built-in` en la *standard-library* de Python.

* De memoria con: [memory_profiler](https://pypi.org/project/memory-profiler/).

## Medición de tiempos

El primer acercamiento que usamos en la nota para perfilar nuestro código es identificar qué es lento, otras mediciones son la cantidad de RAM, el I/O en disco o network. 

### 1) Uso de `time`

In [2]:
import time

### Regla compuesta del rectángulo

**Ejemplo de implementación de regla compuesta de rectángulo: usando math**

Utilizar la regla compuesta del rectángulo para aproximar la integral $\int_0^1e^{-x^2}dx$ con $10^6$ puntos.

In [3]:
f=lambda x: math.exp(-x**2) #using math library

In [4]:
def Rcf(f,a,b,n): #Rcf: rectángulo compuesto para f
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n and h_hat=(b-a)/n
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf (float) 
    """
    h_hat=(b-a)/n
    nodes=[a+(i+1/2)*h_hat for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res

In [5]:
n=10**6

In [6]:
start_time = time.time()
aprox=Rcf(f,0,1,n)
end_time = time.time()

In [7]:
secs = end_time-start_time
print("Rcf tomó",secs,"segundos" )

Rcf tomó 0.3470885753631592 segundos


**Obs:** recuérdese que hay que evaluar que se esté resolviendo correctamente el problema. En este caso el error relativo nos ayuda

In [8]:
def err_relativo(aprox, obj):
    return math.fabs(aprox-obj)/math.fabs(obj) #obsérvese el uso de la librería math

In [9]:
obj, err = quad(f, 0, 1)
err_relativo(aprox,obj)

6.71939731300312e-14

**Comentarios:**

* Tómese en cuenta que siempre hay variación en el tiempo de ejecución. Debe en este caso considerarse tal variación normal al medir los tiempos o pueden construirse ideas erróneas debido a la variación aleatoria en la toma de tiempo.

* También considérese que la máquina en la que se están corriendo las pruebas puede estar realizando otras tareas mientras se ejecuta el código, por ejemplo acceso a la red, al disco a RAM que son factores que pueden causar variación en el tiempo de ejecución del programa.

* Si se van a realizar reportes de tiempos, es importante indicar las características de la máquina en la que se están haciendo las pruebas, p.ej. Dell E6420 con un procesador Intel Core I7-2720QM (2.20 GHz, 6 MB cache, Quad Core) y 8 GB de RAM en un Ubuntu $13.10$.

### 2) Uso de `/usr/bin/time` de Unix

Para la línea de comando `/usr/bin/time` primero escribimos el siguiente archivo en la ruta de este notebook:

In [11]:
%%file Rcf.py

import math
def Rcf(f,a,b,n): #Rcf: rectángulo compuesto para f
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n and h_hat=(b-a)/n
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf (float) 
    """
    h_hat=(b-a)/n
    nodes=[a+(i+1/2)*h_hat for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res
if __name__=="__main__":
        n=10**6
        f=lambda x: math.exp(-x**2)
        print(Rcf(f,0,1,n))

Writing Rcf.py


Lo siguiente es necesario si no tienen en `/usr/bin/time` el comando

In [12]:
%%bash
sudo apt-get install time

Reading package lists...
Building dependency tree...
Reading state information...
time is already the newest version (1.7-25.1build1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [13]:
%%bash
/usr/bin/time -p python3 Rcf.py #la p es de portabilidad, ver: http://manpages.ubuntu.com/manpages/xenial/man1/time.1.html
                                #para mayor información

0.7468241328124773


real 0.41
user 0.39
sys 0.01


**Comentario:** se obtienen tres valores:

* `real` que mide el wall clock o elapsed time
* `user` que mide la cantidad de tiempo que la CPU gastó en la tarea para funciones que no son del kernel.
* `sys` que mide la cantidad de tiempo que la CPU gastó en funciones a nivel de kernel.
* Si se suma `user` con `sys` se tiene una idea de cuánto tiempo se gastó en la CPU y la diferencia entre este resultado y `real` da una idea de cuánto tiempo se gastó para I/O o también puede dar una idea de la cantidad de tiempo que se ocupó el sistema en correr otras tareas.
* La ventaja de `/usr/bin/time` es que no es específico de Python.
* Este comando incluye el tiempo que le toma al sistema iniciar el ejecutable de python (que puede ser significativo si se inician muchos procesos vs un sólo proceso). En el caso de tener short-running scripts donde el tiempo de inicio es significativo del tiempo total entonces `/usr/bin/time` puede ser una medida útil.

* Se puede utilizar la flag `verbose` para obtener más información:

In [14]:
%%bash
/usr/bin/time --verbose python3 Rcf.py

0.7468241328124773


	Command being timed: "python3 Rcf.py"
	User time (seconds): 0.37
	System time (seconds): 0.04
	Percent of CPU this job got: 87%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 0:00.46
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 41204
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 0
	Minor (reclaiming a frame) page faults: 9079
	Voluntary context switches: 25
	Involuntary context switches: 340
	Swaps: 0
	File system inputs: 16
	File system outputs: 0
	Socket messages sent: 0
	Socket messages received: 0
	Signals delivered: 0
	Page size (bytes): 4096
	Exit status: 0


y una explicación (breve) del output se puede encontrar [aquí](http://manpages.ubuntu.com/manpages/xenial/man1/time.1.html). Para el caso de `Major (requiring I/O)` nos interesa que sea $0$ pues indica que el sistema operativo tiene que cargar páginas de datos del disco pues tales datos ya no residen en RAM (por alguna razón).

### 3) Uso de `timeit`

El módulo de `timeit` es otra forma de medir el tiempo de ejecución en la CPU.

**Nota:** el módulo de `timeit` temporalmente desabilita el garbage collector* de Python (esto es, no habrá desalojamiento en memoria de objetos de Python que no se utilicen). Esto puede impactar con un ejemplo del mundo real si el garbage collector es invocado en tus operaciones.

*sugiero buscar qué es el garbage collector en blogs, por ejemplo: [liga](https://rushter.com/blog/python-garbage-collector/) o [liga2](https://stackify.com/python-garbage-collection/) o [liga3](https://stackoverflow.com/questions/4484167/python-garbage-collector-documentation).

## Uso de %timeit

In [15]:
%timeit?

Docstring:
Time execution of a Python statement or expression

Usage, in line mode:
  %timeit [-n<N> -r<R> [-t|-c] -q -p<P> -o] statement
or in cell mode:
  %%timeit [-n<N> -r<R> [-t|-c] -q -p<P> -o] setup_code
  code
  code...

Time execution of a Python statement or expression using the timeit
module.  This function can be used both as a line and cell magic:

- In line mode you can time a single-line statement (though multiple
  ones can be chained with using semicolons).

- In cell mode, the statement in the first line is used as setup code
  (executed but not timed) and the body of the cell is timed.  The cell
  body has access to any variables created in the setup code.

Options:
-n<N>: execute the given statement <N> times in a loop. If <N> is not
provided, <N> is determined so as to get sufficient accuracy.

-r<R>: number of repeats <R>, each consisting of <N> loops, and take the
best result.
Default: 7

-t: use time.time to measure the time, which is the default on Unix.
This f

In [16]:
%timeit -n 5 -r 10 Rcf(f,0,1,n)

333 ms ± 10 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


---

## Uso de line_profiler en python: %lprun

In [11]:
def Rcf_2(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
    Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf_2 (float) 
    """
    mid_point_formula=lambda a,b:a+(b-a)/2.0
    h_hat=(b-a)/n
    nodes=(mid_point_formula(a+h_hat*i, a+h_hat*(i+1)) for i in range(0,n))
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res

In [12]:
def Rcf_3(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
    Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf_3 (float) 
    """
    mid_point_formula=lambda a,b:a+(b-a)/2.0
    h_hat=(b-a)/n
    nodes=(mid_point_formula(a+h_hat*i, a+h_hat*(i+1)) for i in range(0,n))
    return h_hat*sum((f(node) for node in nodes))

In [13]:
def Rcf_4(a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
    Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf_4 (float) 
    """
    mid_point_formula=lambda a,b:a+(b-a)/2.0
    h_hat=(b-a)/n
    nodes=(mid_point_formula(a+h_hat*i, a+h_hat*(i+1)) for i in range(0,n))
    return h_hat*sum((math.exp(-node**2) for node in nodes))

In [14]:
%load_ext line_profiler

In [15]:
%lprun -f Rcf_1 Rcf_1(f,0,1,10**5)

Timer unit: 1e-06 s

Total time: 0.287286 s
File: <ipython-input-3-8b2da2b2416f>
Function: Rcf_1 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def Rcf_1(f,a,b,n):
     2                                               """
     3                                               Compute numerical approximation using rectangle or mid-point method in 
     4                                               an interval.
     5                                               Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
     6                                               Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
     7                                               Args:
     8                                                   f (lambda expression): lambda expression of integrand
     9                                                   a (int): lef

In [16]:
%lprun -f Rcf_2 Rcf_2(f,0,1,10**5)

Timer unit: 1e-06 s

Total time: 0.254116 s
File: <ipython-input-11-08e2d3f927ed>
Function: Rcf_2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def Rcf_2(f,a,b,n):
     2                                               """
     3                                               Compute numerical approximation using rectangle or mid-point method in 
     4                                               an interval.
     5                                               Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
     6                                               Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
     7                                               Args:
     8                                                   f (lambda expression): lambda expression of integrand
     9                                                   a (int): le

In [17]:
%lprun -f Rcf_3 Rcf_3(f,0,1,10**5)

Timer unit: 1e-06 s

Total time: 0.183197 s
File: <ipython-input-12-e6fccc57a114>
Function: Rcf_3 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def Rcf_3(f,a,b,n):
     2                                               """
     3                                               Compute numerical approximation using rectangle or mid-point method in 
     4                                               an interval.
     5                                               Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
     6                                               Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
     7                                               Args:
     8                                                   f (lambda expression): lambda expression of integrand
     9                                                   a (int): le

In [18]:
%lprun -f Rcf_4 Rcf_4(0,1,10**5)

Timer unit: 1e-06 s

Total time: 0.166477 s
File: <ipython-input-13-f3d4bfb56109>
Function: Rcf_4 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def Rcf_4(a,b,n):
     2                                               """
     3                                               Compute numerical approximation using rectangle or mid-point method in 
     4                                               an interval.
     5                                               Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
     6                                               Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
     7                                               Args:
     8                                                   f (lambda expression): lambda expression of integrand
     9                                                   a (int): left

In [19]:
%%file Rcf_1_b.py

import math
@profile
def Rcf_1(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
    Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf_1 (float) 
    """
    mid_point_formula=lambda a,b:a+(b-a)/2.0
    h_hat=(b-a)/n
    nodes=[mid_point_formula(a+h_hat*i, a+h_hat*(i+1)) for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res 
if __name__=="__main__":
        f=lambda x: math.exp(-x**2)
        print(Rcf_1(f,0,1,10**5))

Writing Rcf_1_b.py


**Ejemplo line_profiler desde la línea de comandos:**

In [20]:
%%bash
/home/miuser/.local/bin/kernprof -l -v Rcf_1_b.py

0.7468241328154887
Wrote profile results to Rcf_1_b.py.lprof
Timer unit: 1e-06 s

Total time: 0.230992 s
File: Rcf_1_b.py
Function: Rcf_1 at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           @profile
     4                                           def Rcf_1(f,a,b,n):
     5                                               """
     6                                               Compute numerical approximation using rectangle or mid-point method in 
     7                                               an interval.
     8                                               Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
     9                                               Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
    10                                               Args:
    11                                                   f (lambda expression): l

## Uso de memory_profiler: %memit y %mprun

In [31]:
%load_ext memory_profiler

In [32]:
%memit #how much RAM this process is consuming

peak memory: 97.20 MiB, increment: 0.00 MiB


In [33]:
%memit Rcf_1(f,0,1,10**5)

peak memory: 99.00 MiB, increment: 1.80 MiB


In [34]:
%memit

peak memory: 97.51 MiB, increment: 0.00 MiB


In [35]:
%memit Rcf_2(f,0,1,10**5)

peak memory: 97.51 MiB, increment: 0.00 MiB


In [36]:
%%file Rcf_1_c.py

def Rcf_1(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
    Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf_1 (float) 
    """
    mid_point_formula=lambda a,b:a+(b-a)/2.0
    h_hat=(b-a)/n
    nodes=[mid_point_formula(a+h_hat*i, a+h_hat*(i+1)) for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res 


Writing Rcf_1_c.py


In [37]:
from Rcf_1_c import Rcf_1 as Rcf_1_c

In [38]:
%mprun -f Rcf_1_c Rcf_1_c(f,0,1,10**5)

Filename: /datos/MNO_desde_2018/ramas_repo/mno-master/temas/I.computo_cientifico/Rcf_1_c.py

Line #    Mem usage    Increment   Line Contents
     2     97.5 MiB     97.5 MiB   def Rcf_1(f,a,b,n):
     3                                 """
     4                                 Compute numerical approximation using rectangle or mid-point method in 
     5                                 an interval.
     6                                 Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
     7                                 Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
     8                                 Args:
     9                                     f (lambda expression): lambda expression of integrand
    10                                     a (int): left point of interval
    11                                     b (int): right point of interval
    12                                     n (int): number 

## Uso de cProfile

In [39]:
%%file Rcf_1_a.py
import math
def Rcf_1(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
    Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf_1 (float) 
    """
    mid_point_formula=lambda a,b:a+(b-a)/2.0
    h_hat=(b-a)/n
    nodes=[mid_point_formula(a+h_hat*i, a+h_hat*(i+1)) for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res 
if __name__=="__main__":
        f=lambda x: math.exp(-x**2)
        print(Rcf_1(f,0,1,10**5))

Writing Rcf_1_a.py


In [40]:
%%bash
python3 -m cProfile -s cumulative Rcf_1_a.py

0.7468241328154887
         300067 function calls in 0.091 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.091    0.091 {built-in method builtins.exec}
        1    0.003    0.003    0.091    0.091 Rcf_1_a.py:1(<module>)
        1    0.014    0.014    0.088    0.088 Rcf_1_a.py:2(Rcf_1)
        1    0.030    0.030    0.042    0.042 Rcf_1_a.py:18(<listcomp>)
   100000    0.023    0.000    0.032    0.000 Rcf_1_a.py:24(<lambda>)
   100000    0.011    0.000    0.011    0.000 Rcf_1_a.py:16(<lambda>)
   100000    0.008    0.000    0.008    0.000 {built-in method math.exp}
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:966(_find_and_load)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:936(_find_and_load_unlocked)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:651(_load_unlocked)
        1    0.000    0.000    0.

**Ejercicios**

1. Resuelve los ejercicios y preguntas de la nota.


**Referencias**

1. M. Gorelick, I. Ozsvald, High Performance Python, O'Reilly Media, 2014.